In [48]:
import numpy as np
import pandas as pd

import re
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer

In [49]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/anand/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/anand/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [31]:
data = pd.read_csv("Data_Final")
data.head()

,Unnamed: 0,User_id,Bus_id,Star,Useful,Cool,Funny,Review,State,City,Bus_Ave_Star,User_Review_count,User_Useful_count,User_Funny_count,User_Cool_count,Elite,User_Fans,Users_Ave_Star
0,0,4hBhtCSgoxkrFgHa4YAD-w,bbEXAEFr4RYHLlZ-HFssTA,5.0,0,0,0,"Great burgers,fries and salad! Burgers have a...",CA,Goleta,4.0,922,1687,694,1070,"2015,2016,2017,2018,2019,20,20,2021",51,4.20
1,1,JYYYKt6TdVA4ng9lLcXt_g,SZU9c8V2GuREDN5KgyHFJw,5.0,0,0,0,We were a bit weary about trying the Shellfish...,CA,Santa Barbara,4.0,338,800,144,353,"2012,2013,2014,2015,2016,2017,2018,2019,20,20,...",30,4.12
2,2,Uk3X2AypU8AqvcYEVf7s6Q,eL4lyE7LNoXEMvpcJ8WNVw,3.0,5,0,0,This was a tough one! On the merits of the w...,CA,Santa Barbara,4.0,431,2126,1245,1476,"2012,2013,2014,2015,2016,2017,2018,2019,20,20,...",76,4.05
3,3,LcqNuhqaYt5ekKzaRirmIg,SZU9c8V2GuREDN5KgyHFJw,5.0,2,1,0,"I love trying fresh seafood on piers, wharfs a...",CA,Santa Barbara,4.0,258,452,125,183,"2015,2016,2017",37,3.99
4,4,gasLVm0KRwrVhPGRcqATjw,CHh0ZFrQcsk4boOItr2Zuw,4.0,1,2,0,I stopped in because I was hungry for some sna...,CA,Carpinteria,3.0,1638,2658,933,2297,"2014,2015,2016,2017,2018,2019,20,20,2021",105,3.79


In [32]:
data.shape

(53845, 18)

In [33]:
data["City"].value_counts()

Santa Barbara                       42501
Goleta                               6009
Carpinteria                          2557
Isla Vista                           1425
Montecito                             848
Summerland                            406
Santa  Barbara                         20
Santa Clara                            15
Truckee                                13
Mission Canyon                         10
Cerritos                               10
Santa Barbara                           6
Real Goleta                             4
Santa Barbra                            4
Santa Maria                             3
Los Angeles                             3
Sparks                                  2
Salinas                                 2
Kings Beach                             1
West Hill                               1
Aliso Viejo                             1
Santa Barbara & Ventura Counties        1
Port Hueneme                            1
South Lake Tahoe                  

In [ ]:
data["Star"].value_counts()

5.0    21388
4.0    18655
3.0     8445
2.0     3460
1.0     1897
Name: Star, dtype: int64

In [34]:
data["Star"].value_counts()[1] + data["Star"].value_counts()[2] + data["Star"].value_counts()[3]

13802

In [42]:
data["Star"].value_counts()[4] + data["Star"].value_counts()[5]

40043

In [46]:
documents = []
X = data.Review
stemmer = WordNetLemmatizer()
for i in range(0, len(X)):
    # Remove all the special characters, like parathesis
    document = re.sub(r'\W', ' ', str(X[i]))
    # remove all single characters: like a, b, c, d
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    # Converting to Lowercase
    document = document.lower()
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    documents.append(document)

In [60]:
vectorizer = CountVectorizer(max_features=1500, min_df=0.1, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [61]:
X.shape

(53845, 73)

In [62]:
#------------------------------------Sentiment Lexicon------------------------------------------\
f_n = open("negative-words.txt", "r")
Negative_words = f_n.readlines()
f_p = open("positive-words.txt", "r")
Positive_words = f_p.readlines()
Voca = [i.rstrip('\n') for i in Positive_words] + [i.rstrip('\n') for i in Negative_words]
vectorizer = CountVectorizer(vocabulary=np.unique(Voca))
X = vectorizer.fit_transform(documents).toarray()

In [63]:
X.shape

(53845, 6786)